### LangChain Expression Language

In [ ]:
from dotenv import load_dotenv
load_dotenv()

Old way

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI()

In [ ]:
prompt = ChatPromptTemplate.from_template("tell me about the nutritional value of {input}")

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)
chain.predict(input="Pizza")

New way

In [ ]:
chain = prompt | llm
chain.invoke({"input": "Spaghetti"})

In [ ]:
prompt.input_schema.schema()

In [ ]:
chain.input_schema.schema()

### First pipe

In [ ]:
from langchain.schema.output_parser import StrOutputParser

chain = prompt | llm | StrOutputParser()

In [ ]:
chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "Lasagna"})

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "tell me 5 jokes about {input}"
    )

In [ ]:
chain = prompt | llm.bind(stop=["\n"]) | StrOutputParser()

In [ ]:
chain.invoke({"input": "pizzas"})

### OPENAI Functions

In [ ]:
functions = [
    {
      "name": "joke",
      "description": "A joke",
      "parameters": {
        "type": "object",
        "properties": {
          "setup": {
            "type": "string",
            "description": "The setup for the joke"
          },
          "punchline": {
            "type": "string",
            "description": "The punchline for the joke"
          }
        },
        "required": ["setup", "punchline"]
      }
    }
  ]

In [ ]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
chain = (
    prompt
    | llm.bind(function_call={"name": "joke"}, functions= functions)
    | JsonOutputFunctionsParser()
)

In [ ]:
chain.invoke(input={"input": "bears"})


### Working with vectorstores

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough


vectorstore = Chroma.from_texts(["Cats are typically 9.1 kg in weight.",
                                 "Cats have retractable claws.",
                                 "A group of cats is called a clowder.",
                                 "Cats can rotate their ears 180 degrees.",
                                 "The world's oldest cat lived to be 38 years old."],
                                embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("how old is the oldest cat?")

In [ ]:
from operator import itemgetter

In [ ]:
question = {"bla": "test", "x": "hi"}
itemgetter("bla")


In [ ]:
get_bla = itemgetter("bla")
get_bla(question)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = {
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question"),
    "language": itemgetter("language")
} | prompt | llm | StrOutputParser()

# chain = {
#     "context": (lambda x: x["question"]) | retriever,
#     "question": (lambda x: x["question"]),
#     "language": (lambda x: x["language"])
# } | prompt | llm | StrOutputParser()


In [ ]:
chain.invoke({"question": "how old is the oldest cat?", "language": "german"})

In [ ]:
!pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [ ]:
template = """turn the following user input into a search query for a search engine:

{input}"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = prompt | llm | StrOutputParser() | search

In [ ]:
chain.invoke({"input": "whats the name of the oldest cat?"})

### Arbitrary functions

In [ ]:
from langchain.schema.runnable import RunnableLambda

def length_function(text):
    return len(text)

def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = {
    "a": itemgetter("foo") | RunnableLambda(length_function),
    "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function)
} | prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({"foo": "bar", "bar": "gah"})

### Interface

In [ ]:
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

In [ ]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="")

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

In [ ]:
async for s in chain.astream({"topic": "bears"}):
    print(s.content, end="")
await chain.ainvoke({"topic": "bears"})
await chain.abatch([{"topic": "bears"}])

### Parallelism

In [ ]:
from langchain.schema.runnable import RunnableParallel
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = ChatPromptTemplate.from_template("tell me a bad joke about {topic}") | model
combined = RunnableParallel(joke=chain1, poem=chain2)

In [ ]:
chain1.invoke({"topic": "bears"})

In [ ]:
chain2.invoke({"topic": "bears"})

In [ ]:
combined.invoke({"topic": "bears"})

### Fallbacks

In [ ]:
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a funny comedian and provide funny jokes about specific topics"),
        ("human", "Make a joke about {input}"),
    ]
)

fallback_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You tell the user that you currently are not able to make jokes since you are too tired"),
        ("human", "Make a joke about {input}"),
    ]
)

bad_llm = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | bad_llm | StrOutputParser()

In [ ]:
llm = ChatOpenAI()
good_chain = fallback_prompt | llm

In [ ]:
chain = bad_chain.with_fallbacks([good_chain])

In [ ]:
chain.invoke({"input": "cow"})